In [27]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [28]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for scraping
url = "https://www.royallepage.ca/en/search/homes/on/north-york/?search_str=Ontario%2C+ON%2C+CAN&csrfmiddlewaretoken=8hSYi5r4CNJ4FpajFBKuCoiVy2nAYDnjpx6CO1femOMAv0KMuCiriabofGuDqP9d&property_type=7%2C8&house_type=&features=&listing_type=&lat=50.926163435000035&lng=-84.74492999999995&upper_lat=&upper_lng=&lower_lat=&lower_lng=&bypass=&radius=&zoom=&display_type=gallery-view&travel_time=&travel_time_min=30&travel_time_mode=drive&travel_time_congestion=&da_id=&segment_id=&tier2=False&tier2_proximity=0.2&address=Ontario&method=homes&address_type=province&city_name=&prov_code=ON&school_id=&min_price=0&max_price=2000000&min_leaseprice=0&max_leaseprice=5000%2B&beds=0&baths=2&transactionType=SALE&sfproperty_type%5B7%5D=7&sfproperty_type%5B8%5D=8&keyword=&sortby="

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [29]:
# Go through all pages to collect data
house_listings = []
for x in range(1, 23):
    html = browser.html
    soup = bs(html, 'html.parser')
    links = soup.find_all(class_='address-1')
    
    # Get all address listings for buying    
    print('Gathering data from page:', x)
    for link in links:
        house_listings.append(link.a['href'])
    browser.find_by_css('.next').click()

# Number of listings
len(house_listings)


Gathering data from page: 1
Gathering data from page: 2
Gathering data from page: 3
Gathering data from page: 4
Gathering data from page: 5
Gathering data from page: 6
Gathering data from page: 7
Gathering data from page: 8
Gathering data from page: 9
Gathering data from page: 10
Gathering data from page: 11
Gathering data from page: 12
Gathering data from page: 13
Gathering data from page: 14
Gathering data from page: 15
Gathering data from page: 16
Gathering data from page: 17
Gathering data from page: 18
Gathering data from page: 19
Gathering data from page: 20
Gathering data from page: 21
Gathering data from page: 22


1011

In [30]:
browser.quit()

In [31]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)


# Visit each listing and grab relevant data
house_listing_info = []
for i in range(len(house_listings)):
    url = house_listings[i]
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    houses = {} 

    # Gather information on living space
    living_space_section = soup.find('div', class_='property-meta-info')

    # Get exact address
    address = living_space_section.find('div', class_='title--h2 u-no-margins').text


    # Get price
    price = living_space_section.find('span', class_="title title--h1 price").text.strip()

    
    # Get number of beds
    bed_info = living_space_section.find('div', class_='bed-bath-box__item beds')
    bed_count = bed_info.find('p', class_='bed-bath-box__value').text

    # Get number of baths
    bath_info = living_space_section.find('div', class_='bed-bath-box__item baths')
    bath_count = bath_info.find('p', class_='bed-bath-box__value').text
    
    
    # House features
    building_feature_section = soup.find('div', class_="details-row")
    values = building_feature_section.find_all('span', class_="value")
    
    # Style of house
    style_type = values[0].text.strip(":")

    houses['address'] = address
    houses['price'] = price
    houses['number_of_beds'] = bed_count
    houses['number_of_baths'] = bath_count
    houses['style'] = style_type
    
    
    house_listing_info.append(houses)
    
browser.quit()


AttributeError: 'NoneType' object has no attribute 'find'

In [32]:
house_listing_info

[{'address': '#903 -151 AVENUE RD, Toronto, Ontario, M5R 2H7',
  'price': '$1,955,000',
  'number_of_beds': '2+1 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Apartment'},
 {'address': '7575 4TH LINE, Centre Wellington, Ontario, N0B 1S0',
  'price': '$1,799,900',
  'number_of_beds': '5 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Detached'},
 {'address': '388A WHITMORE AVE, Toronto, Ontario, M6E 2N4',
  'price': '$1,795,000',
  'number_of_beds': '4+2 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Triplex'},
 {'address': '332 DURIE ST, Toronto, Ontario, M6S 3G3',
  'price': '$1,699,000',
  'number_of_beds': '4 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Detached'},
 {'address': '71 WALMSLEY BLVD, Toronto, Ontario, M4V 1X7',
  'price': '$1,599,000',
  'number_of_beds': '3 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Duplex'},
 {'address': '#319 -9909 PINE VALLEY DR, Vaughan, Ontario, L4H 4M1',
  'price': '$1,488,800',
  'number_of_beds': '2 Beds',
  'number_of

In [33]:
import csv

try:
    with open('./Resources/housing.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['address', 'price', 'number_of_beds', 'number_of_baths', 'style'])
        writer.writeheader()
        for data in house_listing_info:
            writer.writerow(data)
except IOError:
    print("I/O error")